# Sequence model  on Gossip data set 

The first model would be only for the title column with a binary target Fake/Not Fake.
This model is done by Andrii Shchur, 2020
Here is its [article](https://towardsdatascience.com/fake-news-detector-with-deep-learning-approach-part-i-eda-757f5c052)

Complete dataset cannot be distributed because of Twitter privacy policies and news publisher copy rights. Social engagements and user information are not disclosed because of Twitter Policy.  

The minimalistic version of latest dataset provided in this repo (located in dataset folder) include following files:
gossipcop_fake.csv - Samples related to fake news collected from GossipCop
gossipcop_real.csv - Samples related to real news collected from GossipCop
Each of the above CSV files is comma separated file and have the following columns

id - Unique identifider for each news
url - Url of the article from web that published that news
title - Title of the news article
tweet_ids - Tweet ids of tweets sharing the news. This field is list of tweet ids separated by tab.

<img src='https://media.giphy.com/media/l0Iyau7QcKtKUYIda/giphy.gif'>

### <span style="background-color: #FFFF00">In this notebook, I have corrected the preprocessing of the data</span>

## Importing packages 

In [1]:
# Load libraries
import pandas as pd
import numpy as np
import os.path as op
import time
import matplotlib.pyplot as plt
#Loading tensorflow and keras 
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from keras.utils import plot_model
#Loading nltk
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
#Loading sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, roc_auc_score
#Loading K-folds
from sklearn.model_selection import StratifiedKFold
#Loading the model
import sys  
sys.path.insert(0, '../../models')
from PY.model5 import get_model

Using TensorFlow backend.


## Paths 

In [2]:
DATA_DIR = '../../../../Data'
CODE_DIR_H5='../../models/H5'
# Create a path for the saving location of the model
MODEL_DIR_H5 = CODE_DIR_H5 + '/model5_Gossip.h5'

## Loading Data 

In [3]:
# Load train data
train = pd.read_csv(op.join(DATA_DIR, 'Gossip/Claims_to_work_with/claims.csv'))
train

author  \
0      https://www.brides.com/story/teen-mom-jenelle-...   
1      https://www.dailymail.co.uk/tvshowbiz/article-...   
2            https://en.wikipedia.org/wiki/Quinn_Perkins   
3      https://www.refinery29.com/en-us/2018/03/19192...   
4      https://www.cnn.com/2017/10/04/entertainment/c...   
...                                                  ...   
22135  www.huffingtonpost.com/2012/09/11/september-11...   
22136  www.dailymail.co.uk/news/article-4915674/NASCA...   
22137  www.telegraph.co.uk/men/the-filter/7-signs-dav...   
22138  www.vanityfair.com/style/2016/09/ryan-gosling-...   
22139  www.lifeandstylemag.com/posts/jamie-foxx-katie...   

                                                   title  label  
0      Teen Mom Star Jenelle Evans' Wedding Dress Is ...      0  
1      Kylie Jenner refusing to discuss Tyga on Life ...      0  
2                                          Quinn Perkins      0  
3      I Tried Kim Kardashian's Butt Workout & Am For...      0  
4      Celine Dion donates concert proceeds to Vegas ...      0  
...                                                  ...    ...  
22135   September 11: Celebrities Remember 9/11 (TWEETS)      1  
22136  NASCAR owners threaten to fire drivers who pro...      1  
22137  The 7 signs that David Beckham is definitely h...      1  
22138  Ryan Gosling and Eva Mendes Did Not Get Marrie...      1  
22139  Jamie Foxx Spends the Day With Katie Holmes an...      1  

[22140 rows x 3 columns]

In [4]:
# Find Na 
train.isnull().sum()

author    269
title       0
label       0
dtype: int64

In [5]:
#Get the Depndent feature
X_train=train.drop('label',axis=1)
y_train=train['label']

In [6]:
# set vocabulary size
vo_size=500
messages=X_train.copy()
messages.reset_index(inplace=True)

In [7]:
len(messages), len(y_train)

(22140, 22140)

In [8]:
# Dataset Preprocessing
cmp=0
ps =PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print("Status: %s / %s" %(i, len(messages)), end="\r")
    review = re.sub('[^a-zA-Z]', ' ',messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    if review :
        review = ' '.join(review)
        corpus.append(review)
        y_train[i]=y_train[i]
    else :
        cmp=cmp+1
        y_train[i]=np.nan
y_train.reset_index()

C:\Users\CBS Compter\anaconda3\envs\tensorflow7\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


Status: 7463 / 22140

C:\Users\CBS Compter\anaconda3\envs\tensorflow7\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\CBS Compter\anaconda3\envs\tensorflow7\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


index  label
0          0    0.0
1          1    0.0
2          2    0.0
3          3    0.0
4          4    0.0
...      ...    ...
22135  22135    1.0
22136  22136    1.0
22137  22137    1.0
22138  22138    1.0
22139  22139    1.0

[22140 rows x 2 columns]

In [9]:
print('the number of lines that could not be preprocessed',cmp)

the number of lines that could not be preprocessed 1


In [10]:
print("We need to delete those",y_train.isnull().sum(),"lines from the label column")
y_train=y_train.dropna()
y_train.astype('Int64')

We need to delete those 1 lines from the label column


0        0
1        0
2        0
3        0
4        0
        ..
22135    1
22136    1
22137    1
22138    1
22139    1
Name: label, Length: 22139, dtype: Int64

In [11]:
# one hot representation
onehot_rep = [one_hot(words, vo_size) for words in corpus]

In [12]:
# pad_sequences
sent_length = 68
embedded_doc=pad_sequences(onehot_rep, padding='pre', maxlen=sent_length)
print(embedded_doc)

[[  0   0   0 ...  28 313 392]
 [  0   0   0 ... 334 167  93]
 [  0   0   0 ...   0 236 266]
 ...
 [  0   0   0 ... 156 138 416]
 [  0   0   0 ... 249 292 340]
 [  0   0   0 ... 335 353 279]]


In [13]:
embedding_vector_feature = 10

In [14]:
# check shape 
len(embedded_doc),y_train.shape

(22139, (22139,))

In [15]:
# final data for NN
X_final=np.array(embedded_doc)
y_final=np.array(y_train)
X_final.shape,y_final.shape

((22139, 68), (22139,))

## Training  the model 

In [ ]:
start=time.time()
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)
# define 10-fold cross validation test harness
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
cvscores1 = []
cvscores2 = []
cvscores3 = []
cvscores4 = []
num_iter=0
for train, test in kfold.split(X_final, y_final):
    num_iter=num_iter+1
    model=get_model(vo_size,embedding_vector_feature,sent_length)
    # Fit the model
    history=model.fit(X_final[train], y_final[train], epochs=10, batch_size=64,verbose=0)
    # evaluate the model
    scores = model.evaluate(X_final[test], y_final[test], verbose=0)
    print("------subset number", num_iter,"------")
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores1.append(scores[1] * 100)
    print("%s: %.2f%%" % (model.metrics_names[2], scores[2]*100))
    cvscores2.append(scores[2] * 100)
    print("%s: %.2f%%" % (model.metrics_names[3], scores[3]*100))
    cvscores3.append(scores[3] * 100)
    print("%s: %.2f%%" % (model.metrics_names[4], scores[4]*100))
    cvscores4.append(scores[4] * 100)
print("--------------------------------------------" )   
print("accuracy","%.2f%% (+/- %.2f%%)" % (np.mean(cvscores1), np.std(cvscores1)))
print("precision","%.2f%% (+/- %.2f%%)" % (np.mean(cvscores2), np.std(cvscores2)))
print("recall","%.2f%% (+/- %.2f%%)" % (np.mean(cvscores3), np.std(cvscores3)))
print("f1-mesure","%.2f%% (+/- %.2f%%)" % (np.mean(cvscores4), np.std(cvscores4)))
end=time.time()

In [ ]:
# Save the model
model.save(MODEL_DIR_H5)
model.summary()

In [ ]:
print('time of training' , end-start)

## Evaluation metrics

In [ ]:
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.show()

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
# summarize history for f1-mesure
plt.plot(history.history['f1_mesure'])
plt.title('model F1-mesure')
plt.ylabel('F1-mesure')
plt.xlabel('epoch')
plt.show()

In [ ]:
# summarize history for precision
plt.plot(history.history['precision_mesure'])
plt.title('model precision')
plt.ylabel('precision')
plt.xlabel('epoch')
plt.show()

In [ ]:
# summarize history for recall
plt.plot(history.history['recall_mesure'])
plt.title('model recall')
plt.ylabel('recall')
plt.xlabel('epoch')
plt.show()